<a href="https://colab.research.google.com/github/Jackelyneg/covid-19-image-classification/blob/main/flask_integrationXray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install flask-ngrok
!pip install flask-bootstrap

In [15]:
from flask_ngrok import run_with_ngrok
import numpy as np
import tensorflow as tf
from flask import Flask,jsonify,make_response,request,render_template,flash,redirect,url_for
from werkzeug.utils import secure_filename
from tensorflow.keras.models import load_model
import pickle
import os

In [16]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
UPLOAD_FOLDER = '/content/drive/MyDrive/Colab Notebooks/project4/static'
ALLOWED_EXTENSIONS = {'jpg','jpeg'}


In [22]:



from flask import Flask, request, jsonify, render_template,url_for
import pickle

from flask_ngrok import run_with_ngrok
app = Flask(__name__, template_folder="/content/drive/MyDrive/Colab Notebooks/project4/templates",static_folder="/content/drive/MyDrive/Colab Notebooks/project4/static" )
app.config['SESSION_TYPE'] = 'memcached'
app.config['SECRET_KEY'] = 'super secret key'

run_with_ngrok(app)
# @app.route("/")
# def home():
#   return render_template('index.html')


app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
def allowed_file(filename):
    print('.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS)
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
@app.route("/")
def home():
  return render_template('index.html')

@app.route('/page2', methods=['GET', 'POST'])
def upload_file():

    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # If the user does not select a file, the browser submits an
        # empty file without a filename.
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            predictions=download_file(name=filename)
            

            full_filename=url_for('static',filename = filename)

            print(full_filename)
            return render_template("xray.html",predictions_text=predictions,filename=full_filename) 
            #return redirect(url_for('download_file', name=filename))
    return render_template("xray.html")       
       


from flask import send_from_directory

@app.route('/uploads/<name>')
def download_file(name):
      
     return xraypred(name)
#@app.route("/getxraypredictions")
def xraypred(fileName):

  model = load_model("/content/drive/MyDrive/project4data/mymodel.h5")
  from tensorflow.keras.preprocessing import image
  filepath="/content/drive/MyDrive/Colab Notebooks/project4/static"+"/"+fileName
  print(filepath)
 
  image_size = (256, 256,1)
  im = image.load_img(filepath, target_size=image_size, color_mode="grayscale")
  
  from tensorflow.keras.preprocessing.image import img_to_array

  image = img_to_array(im)
  img = image.flatten().reshape( -1,256, 256,1)
  #plt.imshow(img.reshape(256, 256))
  img = 1 - img
  #plt.imshow(img.reshape(256, 256), cmap=plt.cm.Greys)
  predictions = model.predict(img)
  predicted= np.argmax(predictions,axis=1)
  if predicted ==0:
    predicted_string="The person has Covid"
  if predicted ==1:
    predicted_string="the person does not have Covid"
  #print(predicted_string)
  #predictions_string=np.array_str(predicted)
  
    
  return (predicted_string)
# @app.route('/display/<filename>')
# def display_image(filename):
# 	#print('display_image filename: ' + filename)
# 	return redirect(url_for('static', filename=filename), code=301)   












    

  
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a7d2-34-125-207-85.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Sep/2021 23:56:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:56:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Sep/2021 23:56:38] "GET /symptoms HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:56:38] "GET /static/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [20/Sep/2021 23:56:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 23:56:47] "GET /static/css/style.css HTTP/1.1" 404 -
